# 波の屈折と反射

In [ ]:
import numpy as np
from math import cos,pi
from matplotlib import pyplot,animation,rc
from IPython.display import HTML

In [ ]:
s = 1.0     # 波の伝わる速さ
L = 2.0     # 系の長さ
T = 4.0     # シミュレーション時間
n = 200     # 空間刻み数
m = 800     # 時間刻み数
dx = L/n    # 空間刻み幅
dt = T/m    # 時間刻み幅
a = s*dt/dx # α

In [ ]:
# 初期条件
x, y = np.meshgrid(np.linspace(0,L,n,False),np.linspace(0,L,n,False),indexing='ij')
u0 = np.zeros((n,n)) # 初期変位
v0 = np.zeros((n,n)) # 初期速度

In [ ]:
ax = np.zeros((n,n))
ax[:,:] = a
for i in range(n):
    for j in range(n):
        if y[i,j] < 1.25*x[i,j]-0.25*L:
            ax[i,j] = 0.75*a

In [ ]:
# 強制振動
w = 2*pi*10 # 各振動数
def oscillate(k):
    return 0.2*cos(w*dt*k)

In [ ]:
# シミュレーション
u = np.zeros((m,n,n))
u[0,:,:] = u0[:,:]
u[0,0,:] = oscillate(0)
u[1,1:-1,1:-1] = (1-2*ax[1:-1,1:-1]**2)*u[0,1:-1,1:-1]+dt*v0[1:-1,1:-1]+(ax[1:-1,1:-1]**2/2)*(u[0,0:-2,1:-1]+u[0,2:n,1:-1]+u[0,1:-1,0:-2]+u[0,1:-1,2:n])
u[1,0,:] = oscillate(1)
for k in range(2,m):
    u[k,1:-1,1:-1] = 2*(1-2*ax[1:-1,1:-1]**2)*u[k-1,1:-1,1:-1]-u[k-2,1:-1,1:-1]+ax[1:-1,1:-1]**2*(u[k-1,0:-2,1:-1]+u[k-1,2:n,1:-1]+u[k-1,1:-1,0:-2]+u[k-1,1:-1,2:n])
    u[k,0,:] = oscillate(k)

In [ ]:
# 等高線プロット
xmin = 70    # x = 0.7-1.7
xmax = 170+1
ymin = 50    # y = 0.5-1.5
ymax = 150+1
tmin = 180   # t = 0.9-1.8
tmax = 360
def update(k):
    pyplot.cla()
    pyplot.contourf(x[xmin:xmax,ymin:ymax],y[xmin:xmax,ymin:ymax],u[k+tmin,xmin:xmax,ymin:ymax])
    pyplot.plot([0.8,1.6],[0.5,1.5],"b")
    pyplot.title('step='+str(k+tmin))
    pyplot.gca().set_aspect('equal');
fig = pyplot.figure()
movie = animation.FuncAnimation(fig,update,frames=tmax-tmin,interval=50)
rc('animation', html='jshtml')
movie

In [ ]:
# 入射波(1,1)、屈折波(0.9825,-0.1860)、反射波(-0.2195,0.9756)の進行方向を重ねてプロット
t = 360
fig = pyplot.figure()
pyplot.contourf(x[xmin:xmax,ymin:ymax],y[xmin:xmax,ymin:ymax],u[t,xmin:xmax,ymin:ymax])
pyplot.plot([0.8,1.6],[0.5,1.5],"b")
pyplot.plot([1.2-1,1.2],[1.0,1.0],"r")
pyplot.plot([1.2,1.2+0.9825],[1.0,1.0-0.1860],"r")
pyplot.plot([1.2,1.2-0.2195],[1.0,1.0+0.9756],"r")
pyplot.title('step='+str(t))
pyplot.xlim([0.7,1.7])
pyplot.ylim([0.5,1.5])
pyplot.gca().set_aspect('equal');
# pyplot.savefig('refraction.pdf') # ファイルへの保存